In [23]:
import MaterialX as mx

print("MaterialX version:", mx.getVersionString())

doc : mx.Document = mx.createDocument()

stdlib = mx.createDocument()
libFiles = mx.loadLibraries(mx.getDefaultDataLibraryFolders(), mx.getDefaultDataSearchPath(), stdlib)  

defs = stdlib.getNodeDefs()
print("Number of node definitions loaded:", len(defs))


MaterialX version: 1.39.5
Number of node definitions loaded: 803


In [24]:

test_name = "ND_tiledimage_color3"
imagefloat = stdlib.getNodeDef(test_name)
if imagefloat:
    print(f"Node Definition '{test_name}' found.")
    #print(mx.prettyPrint(imagefloat))

    graph = imagefloat.getImplementation()
    if graph:
        if graph.isA(mx.NodeGraph) :
            #print(mx.prettyPrint(graph))
            newGraph = imagefloat.makeFunctionalDefinition()
        
            if False and not newGraph:
                ngname = graph.getNodeDefString()
                qualname = graph.getQualifiedName(imagefloat.getName())
                print("nodedef string:", ngname)
                print("qualified name:", qualname)
                if ngname == qualname:
                    newGraphName = graph.getName()
                    newGraph = imagefloat.addChildOfCategory("nodegraph", newGraphName)
                    if not newGraph:
                        print("Failed to create new functional node graph:", newGraphName)
                    else:   
                        print("Created new functional node graph:", newGraphName)
                        newGraph.copyContentFrom(graph)
                        newGraph.removeAttribute(mx.InterfaceElement.NODE_DEF_ATTRIBUTE)
                        graph.removeAttribute(mx.InterfaceElement.NODE_DEF_ATTRIBUTE)
                        
                        parent = imagefloat.getParent()
                        tempName = parent.createValidChildName(newGraphName + "_old")
                        graph.setName(tempName)
                        newGraph.setName(newGraphName)
                    

        if newGraph:
            print("New functional node definition name:", newGraph.getName())
            print(mx.prettyPrint(imagefloat))

Node Definition 'ND_tiledimage_color3' found.
New functional node definition name: NG_tiledimage_color3_old
<nodedef name="ND_tiledimage_color3" node="tiledimage" nodegroup="texture2d">
  <input name="file" type="filename" value="" uniform="true">
  <input name="default" type="color3" value="0.0, 0.0, 0.0">
  <input name="texcoord" type="vector2" defaultgeomprop="UV0">
  <input name="uvtiling" type="vector2" value="1.0, 1.0">
  <input name="uvoffset" type="vector2" value="0.0, 0.0">
  <input name="realworldimagesize" type="vector2" value="1.0, 1.0" unittype="distance">
  <input name="realworldtilesize" type="vector2" value="1.0, 1.0" unittype="distance">
  <input name="filtertype" type="string" value="linear" enum="closest,linear,cubic" uniform="true">
  <input name="framerange" type="string" value="" uniform="true">
  <input name="frameoffset" type="integer" value="0" uniform="true">
  <input name="frameendaction" type="string" value="constant" enum="constant,clamp,periodic,mirror" un

In [25]:
stdsurf = stdlib.getNodeDef("ND_standard_surface_surfaceshader")
nodegraph_counts = {}
if stdsurf:
    print(f"got standard surface node def: {stdsurf.getVersionString()}")
    other_stdsurf = stdsurf.getMatchingDefinitions()
    print("number of matching definitions:", len(other_stdsurf))
    for ndstring in other_stdsurf:
        print("matching definition:", ndstring)
        nd = stdlib.getNodeDef(ndstring)
        if nd:
            print("  version:", nd.getVersionString())
            impl = nd.getImplementation()
            nodegraph_string = impl.getAttribute("nodegraph")
            if nodegraph_string:
                impl = stdlib.getNodeGraph(nodegraph_string)
                if nodegraph_string in nodegraph_counts:
                    nodegraph_counts[nodegraph_string] += 1
                else:
                    nodegraph_counts[nodegraph_string] = 1
            if impl and impl.isA(mx.NodeGraph):
                print("  implementation:", impl.getName())

print("*"*80)
print("Node graph usage counts among standard surface definitions:")
for ngname, count in nodegraph_counts.items():
    print(f"  {ngname}: {count}")

got standard surface node def: 1.0.1
number of matching definitions: 2
matching definition: ND_standard_surface_surfaceshader
  version: 1.0.1
  implementation: NG_standard_surface_surfaceshader_100
matching definition: ND_standard_surface_surfaceshader_100
  version: 1.0.0
  implementation: NG_standard_surface_surfaceshader_100
********************************************************************************
Node graph usage counts among standard surface definitions:
  NG_standard_surface_surfaceshader_100: 2
